## Context
We notice that the obtained unwrapped phase appears to be invariant from the voltage being displaced from 0.

In [ ]:
# Without the use of a transimpedance amplifier, voltage is always postive. Acts on a trace file.

# . Honours Module Folder
# ├ FYPLibrary
# | ├ file_reading.py
# | └ IQ_demod.py
# └ Sub Project folder i.e.: os.path.abspath('')
#   └ This Jupyter notebook(.ipynb)

# Initialiastion: Directory appending for my system. Vary the directories as necessary.
import sys, os.path
# Add the FYP folder for import
if 'FYPLibrary' not in [os.path.basename(x) for x in sys.path]:
    cur_path = os.path.abspath('')
    while 'FYPLibrary' not in os.listdir(cur_path):
        cur_path = os.path.dirname(cur_path)
        if not cur_path:
            raise ValueError('Recursively climbed up directory past root!')
    sys.path.append(os.path.join(cur_path, 'FYPLibrary'))
    del cur_path

In [ ]:
# from pprint import pprint
# pprint(sys.path)

In [ ]:
# Import Modules
# from os import listdir
from file_reading import *
from IQ_demod import *
import numpy as np
from numpy import pi
from EPstandard import easy_read_popt_pcov as print_popt_pcov
from scipy.optimize import curve_fit
# from scipy.signal import periodogram
import matplotlib.pyplot as plt

In [ ]:
trcPath = "/home/thormund/fibre-sensing/Experiments 2/20220719 Heterodyne Phasemeter Test/traces/C2-140.25-1.0-test00000.trc"

In [ ]:
# Place script specific functions here
def displaced_sin(x, A, f, phi, c):
    return A * np.sin(2*pi*f*x + phi) + c

def get_time_signal_from_trc(file):
    # Read and obtain trace
    NAME = os.path.basename(file)
    if NAME[0] != 'C' and NAME[-4:] != '.txt':
        print(f"[Warning] Unrecognised file reached, Skipping: {NAME}\n")
        return

    # Edit Initialisation
    num_vars = numerical_variables_from_name(NAME)
    SIGNAL_F = 70.125e6*2 #Hz 
    SAMPLING_F = 1.0e6 #Hz
    ph_ad = phase_advance(SIGNAL_F, SAMPLING_F) # phase advance = 2*pi/N
    N, _ = freq_ratio(signal=SIGNAL_F, sample=SAMPLING_F)
    print(f"[Int Debug] {num_vars = } {N = }")
    
    # Lecroy parser
    time_axis, signals = parse_and_read_oscilliscope_trc(file)
    signal = signals[0]

    # Time
    meta = {'Record Length': (len(signal), 'Points'), \
        'Sample Interval': (1/SAMPLING_F, 's'), \
        'Inverse Sample Interval': (SAMPLING_F, 'Hz'), \
        'Trigger Point': ('unknown', 'Samples'), \
        'Trigger Time': ('unknown', 's'), \
        'Horizontal Offset': ('unknown', 's')}
    t_axis = np.arange(start= 0, 
        stop= (int(meta["Record Length"][0])-N+1), 
        step= 1) * meta['Inverse Sample Interval'][0]

    return ph_ad, t_axis, signal, N, meta, time_axis

# For each file in files selected
def get_plot(phases, t_axis):
    # wd is the write directory for results of this given file
    # Edit kwargs as necessary
    
    # Draw figure
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.plot(t_axis, phases, color = 'mediumblue', linewidth= 0.1)
    ax.set_ylabel(r'$\phi$/rad', usetex= True)
    ax.set_xlabel(r'$t$/s', usetex= True)
    plt.title(f"Phase Reconstruction", usetex= True)
    fig.set_size_inches(2.5*(8.25-0.875*2), 6)
    fig.tight_layout()
    plt.show(block= False)

    return

In [ ]:
def process_signal(time_axis, signal):
    # Correct for displace voltage
    popt, pcov = curve_fit(displaced_sin, time_axis, signal,
        p0= [np.max(signal) - np.min(signal), 0.25e6, 0, np.mean(signal)])
    signal -= popt[-1]
    print_popt_pcov(popt, pcov)
    return signal

def get_phases(ph_ad, signal, N):
    # Yield phases
    phases = signal_to_phase(signal, N, ph_ad, phase_advancement_correction= False)
    phases = phase_reconstruction_2(phases, ph_ad)
    
    radii = get_R_signal(signal, N, ph_ad)
    print(f"{np.mean(radii) = }\n{np.std(radii) = }")
    return phases

In [ ]:
ph_ad, t_axis, signal, N, meta, time_axis = get_time_signal_from_trc(trcPath)
def draw_signal(time_axis, signal):
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.plot(time_axis, signal, color = 'mediumblue', linewidth= 0.028)
    ax.set_ylabel(r'$\phi$/rad', usetex= True)
    ax.set_xlabel(r'$t$/s', usetex= True)
    plt.title(f"Signal", usetex= True)
    fig.set_size_inches(2.5*(8.25-0.875*2), 6)
    fig.tight_layout()
    plt.show(block= False)

draw_signal(time_axis, signal)

In [ ]:
correct_signal = process_signal(time_axis, signal)
draw_signal(time_axis, correct_signal)

In [ ]:
phase_uncorrected = get_phases(ph_ad, signal+2, N)
get_plot(phase_uncorrected, t_axis)

In [ ]:
phase_corrected = get_phases(ph_ad, correct_signal, N)
get_plot(phase_corrected, t_axis)

In [ ]:
my_signal = np.sin(
        np.arange(start= 0, 
        stop= 5 * pi, 
        step= pi/2) )

def convolution(dphi, signal):
    # lookup tables
    sines = np.sin([i * dphi for i in range(N)])
    cosines = np.cos([i * dphi for i in range(N)])

    # summation process # rewrite for direct multiplication
    # print(f"[Debug] {signal[0:N] = }")
    # Is = (2/N) * np.fromiter([np.dot(signal[j:j+N], sines) for j in range(len_signal-N)], 
    #     dtype= np.float64, count= len_signal-N)
    # Qs = (2/N) * np.fromiter([np.dot(signal[j:j+N], cosines) for j in range(len_signal-N)], 
    #     dtype= np.float64, count= len_signal-N)
    Is = (2/N) * np.convolve(signal, sines[::-1], mode= 'valid')
    Qs = (2/N) * np.convolve(signal, cosines[::-1], mode= 'valid')
    return Is, Qs

print(convolution(pi/2, my_signal)[0])
print(convolution(pi/2, my_signal+2)[0])